In [1]:
import os

path = "/home/praveen/Dropbox/Masters/emission-mnras-cotar/data"
files = [os.path.splitext(filename)[0] for filename in os.listdir(path)]
print (files)

['1902110022010241', '1902230017012411', '1706030041012351', '1902120016013191', '1611070039013771', '1705080033012212', '1902230017012413', '1705080033012214', '1902100024013022', '1601300036010743', '1601230026010123', '1504290031011624', '1902100016012382', '1902060035010442', '1504290031011623', '1706040021011451', '1902120016013384', '1902110022013603', '1704080050012173', '1403120045011163', '1902240021013511', '1902100024013023', '1704130036010672', '1902110016011483', '1902120016013423', '1902120016013383', '1706040021011452', '1902120016013192', '1601230026010122', '1704130036010673', '1706040021011453', '1504290031011621', '1604220057011512', '1601230026010124', '1902060035010444', '1504290031012681', '1611070039013772', '1902100024010371', '1902120016013194', '1601300036010741', '1902110022013602', '1902120016013513', '1902110022010244', '1902060035010443', '1706040021011454', '1902230017012414', '1704080050012172', '1604220057011514', '1704080050012171', '1504290031012682',

In [2]:
type(files)

list

In [3]:
suffix_list = []

for e in files:
    for element in e.split():
        if element.endswith("3"):
            suffix_list.append(element)

print(len(suffix_list))

25


In [5]:
type(suffix_list[1])

str

In [8]:
suffix_list

['1902230017012413',
 '1601300036010743',
 '1601230026010123',
 '1504290031011623',
 '1902110022013603',
 '1704080050012173',
 '1403120045011163',
 '1902100024013023',
 '1902110016011483',
 '1902120016013423',
 '1902120016013383',
 '1704130036010673',
 '1706040021011453',
 '1902120016013513',
 '1902060035010443',
 '1604220057011513',
 '1902240021013513',
 '1611070039013773',
 '1902100016012383',
 '1706030041012353',
 '1705080033012213',
 '1902120016013193',
 '1902110022010243',
 '1504290031012683',
 '1902100024010373']

In [6]:
new_list = [s.strip("3") for s in suffix_list]

In [7]:
new_list

['190223001701241',
 '160130003601074',
 '160123002601012',
 '150429003101162',
 '190211002201360',
 '170408005001217',
 '140312004501116',
 '190210002401302',
 '190211001601148',
 '190212001601342',
 '190212001601338',
 '170413003601067',
 '170604002101145',
 '190212001601351',
 '190206003501044',
 '160422005701151',
 '190224002101351',
 '161107003901377',
 '190210001601238',
 '170603004101235',
 '170508003301221',
 '190212001601319',
 '190211002201024',
 '150429003101268',
 '190210002401037']

In [9]:
import pandas as pd 
df = pd.DataFrame(new_list)

In [10]:
df.head()

,0
0,190223001701241
1,160130003601074
2,160123002601012
3,150429003101162
4,190211002201360


In [14]:
df.to_csv("GALAH_DR3_sobject_ids.csv", header=False, index=False)

In [17]:
import numpy as np
import pandas as pd 
import astropy.io.fits as pyfits
import matplotlib.pyplot as plt
import glob
from spectres import spectres

import pandas as pd
df = pd.read_csv("best_emission_candidates.csv",header=None)
df.columns = ["sobject_id"]
sobject_ids = df["sobject_id"].to_numpy().tolist()

GRID_SIZE = 0.06
LOWER_LAMBDA = 6472.5
UPPER_LAMBDA = 6740
regrid = np.arange(LOWER_LAMBDA, UPPER_LAMBDA, GRID_SIZE) 

def read_spectra(sobject_id):
    fits_files = [[],[],[],[]]
    for each_ccd in [1,2,3,4]:
        fits_files[each_ccd-1] = glob.glob("/home/praveen/Dropbox/Masters/python-code/galah-file-read-test/data/GALAH_DR3/spectra/SPECTRA/"+str(sobject_id)+str(each_ccd)+".fits") #this is reading fits files from file and not downloading directly 

    spectrum = dict()
    for each_ccd in [1,2,3,4]: #GALAH uses indexing from 1 - 4
        if fits_files[each_ccd-1]!=[]: #just using zero indexing here 
            fits = pyfits.open(fits_files[each_ccd-1][0]) 
            
            # Extension 0: Reduced spectrum
            # Extension 1: Relative error spectrum
            # Extension 4: Normalised spectrum, NB: cut for CCD4

            # Extract wavelength grid for the reduced spectrum
            start_wavelength = fits[0].header["CRVAL1"]
            dispersion       = fits[0].header["CDELT1"]
            nr_pixels        = fits[0].header["NAXIS1"]
            reference_pixel  = fits[0].header["CRPIX1"]

            if reference_pixel == 0:
                reference_pixel = 1
            spectrum['wave_red_'+str(each_ccd)] = ((np.arange(0,nr_pixels)--reference_pixel+1)*dispersion+start_wavelength) #this is the reduced spectrum 

            # Extract wavelength grid for the normalised spectrum
            start_wavelength = fits[4].header["CRVAL1"]
            dispersion       = fits[4].header["CDELT1"]
            nr_pixels        = fits[4].header["NAXIS1"]
            reference_pixel  = fits[4].header["CRPIX1"]

            if reference_pixel == 0:
                reference_pixel=1
            spectrum['wave_norm_'+str(each_ccd)] = ((np.arange(0,nr_pixels)--reference_pixel+1)*dispersion+start_wavelength) #this is the normalised spectrum 

            spectrum['sob_red_'+str(each_ccd)]  = np.array(fits[0].data)
            spectrum['uob_red_'+str(each_ccd)]  = np.array(fits[0].data * fits[1].data)

            spectrum['sob_norm_'+str(each_ccd)] = np.array(fits[4].data)
            if each_ccd != 4:
                spectrum['uob_norm_'+str(each_ccd)] = np.array(fits[4].data * fits[1].data)
            else:
                # for normalised error of CCD4, only used appropriate parts of error spectrum
                spectrum['uob_norm_4'] = np.array(fits[4].data * (fits[1].data)[-len(spectrum['sob_norm_4']):])

            fits.close()
        else:
            spectrum['wave_red_'+str(each_ccd)] = []
            spectrum['wave_norm_'+str(each_ccd)] = []
            spectrum['sob_red_'+str(each_ccd)] = []
            spectrum['sob_norm_'+str(each_ccd)] = []
            spectrum['uob_red_'+str(each_ccd)] = []
            spectrum['uob_norm_'+str(each_ccd)] = []
    
    spectrum['wave_red'] = np.concatenate(([spectrum['wave_red_'+str(each_ccd)] for each_ccd in [1,2,3,4]]))
    spectrum['wave_norm'] = np.concatenate(([spectrum['wave_norm_'+str(each_ccd)] for each_ccd in [1,2,3,4]]))
    spectrum['sob_red'] = np.concatenate(([spectrum['sob_red_'+str(each_ccd)] for each_ccd in [1,2,3,4]]))
    spectrum['sob_norm'] = np.concatenate(([spectrum['sob_norm_'+str(each_ccd)] for each_ccd in [1,2,3,4]]))
    spectrum['uob_red'] = np.concatenate(([spectrum['uob_red_'+str(each_ccd)] for each_ccd in [1,2,3,4]]))
    spectrum['uob_norm'] = np.concatenate(([spectrum['uob_norm_'+str(each_ccd)] for each_ccd in [1,2,3,4]]))

    return spectrum 
    
    #returns a 30 "row" dict of numpy array per row 'wave_red_x' is the key for the key value pair
    #camera 3 will be the more useful canmera for Li spectra 

def resample_spectra(spectrum, camera, verbose):

    spec_resample, spec_errs_resample = spectres(regrid,  spectrum['wave_norm_'+str(camera)], spectrum['sob_norm_'+str(camera)], spec_errs= spectrum['uob_norm_'+str(camera)],verbose=verbose) 

    return spec_resample, spec_errs_resample



In [18]:
df.head()

,sobject_id
0,131116000501262
1,131116001001379
2,131118002401234
3,131118002401396
4,131118002901052


In [19]:
sobject_ids = df["sobject_id"].to_numpy().tolist()

In [ ]:
import h5py

#save the resampled spectra to be used as inputs to the training set
hf_spec = h5py.File("/data/praveen/resampled_emission_spectra.h5", "w")
hf_spec.create_dataset('spectra', data=resampled_spectra_collection['spec_resample'])
hf_spec.close()

#save the wavelength grid 
hf_grid = h5py.File("/data/praveen/wl_grid.h5", "w")
hf_grid.create_dataset('wl_grid', data=regrid)
hf_grid.close()

#save the error spectra
hf_error = h5py.File("/data/praveen/resampled_test_errors.h5", "w")
hf_error.create_dataset('errors', data=resampled_error_collection['error_resample'])
hf_error.close()